<a href="https://colab.research.google.com/github/grant88/education/blob/main/parsing_and_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в заголовке которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [ ]:
import requests
import pandas as pd
import time
import re
from datetime import datetime
from bs4 import BeautifulSoup
from bs4.element import Tag

In [ ]:
# определяем список хабов, которые нам интересны
DESIRED_HUBS = ['python', 'парсинг']

In [ ]:
def get_soup_from_url(url: str) -> BeautifulSoup:
    req = requests.get(url)
    return BeautifulSoup(req.text, 'html.parser')

In [ ]:
def get_post_info(post: Tag) -> dict:
    regex = re.compile('tm-(article|megapost)-snippet__title')
    header = post.find('h2', {'class': regex})
    if header:
        for hub in DESIRED_HUBS:
            if hub in header.text.lower():

                url = f"https://habr.com{header.find('a').get('href')}"
                date = post.find('time').get('datetime')
                post_date = datetime.strptime(date, '%Y-%m-%dT%H:%M:%S.000Z')

                return {
                    'post_date': post_date,
                    'header': header.text,
                    'url': url,
                }

In [ ]:
pages_count = 50
resulted_posts = []
for page_id in range(1, pages_count + 1):
    url = f"http://habr.com/ru/all/page{str(page_id)}/"
    page = get_soup_from_url(url)
    posts = page.find_all('article', {'class':'tm-articles-list__item'})
    for post in posts:
        post_info = get_post_info(post)
        if post_info:
            resulted_posts.append(post_info)
    time.sleep(0.3)

In [ ]:
df = pd.DataFrame(resulted_posts)

In [ ]:
df.head()

,post_date,header,url
0,2022-02-22 08:29:20,Бот-трафик и парсинг цен – взгляд со стороны в...,https://habr.com/ru/company/proto/blog/652861/
1,2022-02-21 19:33:41,Обнаружение новизны изображений с помощью Pyth...,https://habr.com/ru/post/652851/
2,2022-02-20 21:57:45,Машинное стереозрение для новичков: две камеры...,https://habr.com/ru/company/skillfactory/blog/...
3,2022-02-19 18:00:39,Еще один способ развернуть python автотесты we...,https://habr.com/ru/post/652511/
4,2022-02-17 06:57:45,Atlas: Путешествие Dropbox от монолита на Pyth...,https://habr.com/ru/company/flant/blog/651871/


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только заголовки статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`

In [ ]:
def get_post_content(url: str) -> str:
    soup = get_soup_from_url(url)
    time.sleep(0.3)
    return soup.find('article').find('div', id="post-content-body").text

In [ ]:
df['article_text'] = df['url'].apply(get_post_content)

In [ ]:
df.head()

,post_date,header,url,article_text
0,2022-02-22 08:29:20,Бот-трафик и парсинг цен – взгляд со стороны в...,https://habr.com/ru/company/proto/blog/652861/,"В данной статье я хочу рассказать про то, как ..."
1,2022-02-21 19:33:41,Обнаружение новизны изображений с помощью Pyth...,https://habr.com/ru/post/652851/,"В этой статье я расскажу, как с по..."
2,2022-02-20 21:57:45,Машинное стереозрение для новичков: две камеры...,https://habr.com/ru/company/skillfactory/blog/...,Стажируясь в правительственном технологическом...
3,2022-02-19 18:00:39,Еще один способ развернуть python автотесты we...,https://habr.com/ru/post/652511/,Всем привет! Меня зовут Илья. Я люблю писать а...
4,2022-02-17 06:57:45,Atlas: Путешествие Dropbox от монолита на Pyth...,https://habr.com/ru/company/flant/blog/651871/,"Прим. перев.: в этом материале, опубликованном..."


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы. Данные (почтовые адреса) передаются на сервис в json-формате (параметр `json` функции `post`).

In [ ]:
# todo

### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`